# Working with BIDS data

In [3]:
import matplotlib
import pathlib
!pip install mne-bids
import mne
import mne_bids

matplotlib.use('Qt5Agg')

## Read the input data we wish to convert to BIDS

In [4]:
sample_data_dir = mne.datasets.sample.data_path()
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

Opening raw data file C:\Users\Dell\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


C:\Users\Dell\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


320 events found
Event IDs: [ 1  2  3  4  5 32]


## Write the raw data to BIDS!

We need to specify the power line frequency.

In [5]:
# bids require the powerline frequency must be stored somewhere in data
raw.info['line_freq'] = 60

Now, actually write the data.

In [6]:
out_path = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True)

Opening raw data file C:\Users\Dell\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Writing 'out_data\sample_BIDS\README'...
Writing 'out_data\sample_BIDS\participants.tsv'...
Writing 'out_data\sample_BIDS\participants.json'...
Writing 'out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_coordsystem.json'...
Writing 'out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_coordsystem.json'...
Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']
Writing 'out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv'...
Writing 'out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.json'...
Writing 'out_data\sample_BIDS\dataset_description.json'...
Writing 'out

C:\Users\Dell\AppData\Local\Temp\ipykernel_21908\3490163292.py:9: FutureWarning: The events_data parameter has been deprecated in favor the new events parameter, to ensure better consistency  with MNE-Python. The events_data parameter will be removed in MNE-BIDS 0.14. Please use the events parameter instead.
  mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,


Writing 'out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv'...
Copying data files to sub-01_ses-01_task-audiovisual_run-01_meg.fif
Reserving possible split file sub-01_ses-01_task-audiovisual_run-01_split-01_meg.fif
Writing C:\Users\Dell\Desktop\mne\pybrain_mne-main\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif
Closing C:\Users\Dell\Desktop\mne\pybrain_mne-main\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif
[done]
Writing 'out_data\sample_BIDS\sub-01\ses-01\sub-01_ses-01_scans.tsv'...
Wrote out_data\sample_BIDS\sub-01\ses-01\sub-01_ses-01_scans.tsv entry with meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif.


BIDSPath(
root: out_data/sample_BIDS
datatype: meg
basename: sub-01_ses-01_task-audiovisual_run-01_meg.fif)

## Write MEGIN / Elekta / NeuroMag fine-calibration and crosstalk files

In [ ]:
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path=bids_path)


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Write the same raw data again, but this time act as if it were from a different subject – subject "02".</li>
    </ul>
</div>

## Print the created file & directory structure

In [ ]:
mne_bids.print_dir_tree(out_path)

## Generate a data summary

In [ ]:
print(mne_bids.make_report(out_path))

## Reading BIDS data

Let's try to read back the data we just stored according to the BIDS specification.

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

In [ ]:
raw.plot()

## Events are stored as annotations – but we convert between the two.

In [ ]:
raw.annotations[0]

In [ ]:
events, event_id = mne.events_from_annotations(raw)

In [ ]:
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

## Fine the MEG fine-calibration and crosstalk files

In [ ]:
bids_path.meg_calibration_fpath

In [ ]:
bids_path.meg_crosstalk_fpath